In [1]:
import sys

sys.path.append('..')

In [2]:
import sqlite3
from pathlib import Path
from pprint import pp

import pandas as pd
from IPython.display import display
from ipywidgets import interact
from PIL import Image, ImageDraw
from sklearn import metrics

from herbarium.pylib import db

In [3]:
DATA = Path('..') / 'data'

DB = DATA / 'angiosperms.sqlite'
IMAGES = DATA / 'images'

In [4]:
TEST_RUNS = """ b0_flowers_all_orders_1 b0_fruits_all_orders_1 """.split()

FIELDS = [
    ('reproductivecondition', 'reproductive condition'),
    ('occurrenceremarks', 'occurrence remarks'),
    ('fieldnotes', 'field notes'),
    ('dynamicproperties', 'dynamic properties'),
]

TRAITS = [
    ('flowering', 'not_flowering'),
    ('fruiting', 'not_fruiting'),
    # ('leaf_out', 'not_leaf_out'),
]

FLOWERING = db.select_test_run(DB, 'b0_flowers_all_orders_1')

FRUITING = db.select_test_run(DB, 'b0_fruits_all_orders_1')

In [5]:
true_pos = [r for r in FLOWERING if r['true'] == 1 and r['true'] == round(r['pred'])]
true_neg = [r for r in FLOWERING if r['true'] == 0 and r['true'] == round(r['pred'])]
false_pos = [r for r in FLOWERING if r['true'] == 1 and r['true'] != round(r['pred'])]
false_neg = [r for r in FLOWERING if r['true'] == 0 and r['true'] != round(r['pred'])]

In [6]:
y_true = pd.Series([round(r['true']) for r in FLOWERING])
y_pred = pd.Series([round(r['pred']) for r in FLOWERING])

df_confusion = pd.crosstab(y_true, y_pred, rownames=['Actual'], colnames=['Predicted'])
df_confusion

Predicted,0,1
Actual,,
0,186,117
1,10,2286


In [7]:
df_conf_norm = df_confusion / df_confusion.sum(axis=1)
df_conf_norm

Predicted,0,1
Actual,,
0,0.613861,0.050958
1,0.033003,0.995645


In [11]:
dataset = false_neg


def flowering(idx):
    rec = dataset[idx - 1]
    image = Image.open(Path('..') / rec['path'])

    for field, label in FIELDS:
        print(f'{(label+":"):<23} {rec[field]}')

    flag = ''
    if rec['flowering']:
        flag += '1'
    elif rec['not_flowering']:
        flag += '0'
    print(f'\n{"flowering NLP:":<23} {flag}')

    print(f'{"flowering model:":<23} {round(rec["pred"])} ({rec["pred"]:0.4})')

    display(image)


# flowering(0)
interact(flowering, idx=(1, len(dataset)))

interactive(children=(IntSlider(value=59, description='idx', max=117, min=1), Output()), _dom_classes=('widget…

<function __main__.flowering(idx)>